In [1]:
import sys
sys.path.append('../src')

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression # type: ignore
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import RFE
from sklearn.model_selection import cross_val_score

from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier
import matplotlib.pyplot as plt

import pickle

from functions import *
from data_cleaning import DataCleaning

from betfairlightweight import StreamListener, APIClient
import os


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

/Users/andrewbarwise/miniforge3/envs/horse_trading/lib/python3.11/site-packages/pydantic/_internal/_fields.py:151: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/andrewbarwise/miniforge3/envs/horse_trading/lib/python3.11/site-packages/pydantic/_internal/_config.py:322: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [2]:
df = pd.read_csv('../data/raw_proform/2024_10_13.csv')

df.head()

,Race Time,Course,Distance (y),Race Type,Official Going,Classifications,Position,Horse,Jockey Claim,Weight (pounds),SP Odds Decimal,BF Decimal SP,"Won (1=Won, 0=Lost)",PRC Average,PRC Last Run,PRC 2nd Last Run,PRC 3rd Last Run,Pace Rating,Pace Rating Rank,OR,OR Rank,Tissue Odds Decimal,PFR Rank,Main,Main Rank,Main Stats,Stats,Stats Rank,Trn Stats,Jky Stats,TrnJky Stats,Hrs Stats,HA Career Speed Rating,HA Career Speed Rating Rank,HA Last 1 Year Speed Rating,HA Last 1 Year Speed Rating Rank,MR Career Speed Rating,MR Career Speed Rating Rank,MR Last 1 Year Speed Rating,MR Last 1 Year Speed Rating Rank,MR Last 3 Runs Speed Rating,MR Last 3 Runs Speed Rating Rank,LTO Speed Rating,LTO Speed Rating Rank,2nd LTO Speed Rating,2nd LTO Speed Rating Rank,3rd LTO Speed Rating,4th LTOt Speed Rating,Evening Price,Breakfast Price,Morning Price,dyHA,dyLTO,dyAVG,dyAVG(x),dyMR,dyMR(x),dy2HA,Finished Races SR,WON SR Before,Won P/L Before,Plc SR Before,DSLR(F),DSLR(J),FLAT TURF Runs Before,FLAT AW Runs Before,CHASE Runs Before,HURDLE Runs Before,NHF Runs Before,ALL FLAT Runs Before,FLAT TURF Wins Before,FLAT AW Wins Before,CHASE Wins Before,HURDLE Wins Before,NHF Wins Before,Distance In Yards LTO,Official Rating LTO,Distance To Winner LTO,Betfair SP Odds LTO (Decimcal),Speed Rating LTO,PRC Average LTO,PRC Last Run LTO,PRC 2nd Last Run LTO,PRC 3rd Last Run LTO,JSH Mean Movement Rank,Cherry Picked Rating,Today's Going Wins,Today's Going Places,Today's Going Runs,Today's Distance Wins,Today's Distance Places,Today's Distance Runs,Today's Class Wins,Today's Class Places,Today's Class Runs,Today's Course Wins,Today's Course Places,Today's Course Runs,Heavy/Soft Ground Wins,Heavy/Soft Ground Places,Heavy/Soft Ground Runs,Good to Soft Ground Wins,Good to Soft Ground Runs,Good Ground Wins,Good Ground Runs,Good to Firm Ground Wins,Good to Firm Ground Runs,SpeedRating LTO
0,01/06/2023 2:00:00 PM,Yarmouth,1763,Turf,Good To Firm,Hcap,1,Simply Gorgeous,0,136,9.0,10.83,1,39,13,51,53,6,4,56,1,9.5,8,274,5,622,348,1,88,94,91,75,57,8,36,8,52,6,36,8,36,8,36,8,0,5,0,0,8.0,7.0,6.0,63.0,36.0,53.0,51.0,56.0,59.0,60.0,NaN,7.69,-4.50,7.69,29.0,NaN,4,9,0,0,0,13,0,1,0,0,0,1530,57.0,15.600000,8.34,36.0,60.0,51.0,53.0,76.0,NaN,NaN,0,1,3,1,0,7,1,0,7,0,0,0,0,0,0,0,0,0,1,0,3,36.0
1,01/06/2023 2:00:00 PM,Yarmouth,1763,Turf,Good To Firm,Hcap,2,Bold And Loyal,0,126,8.5,10.83,0,43,30,29,71,6,4,46,9,8.5,2,293,3,538,245,8,75,67,66,37,65,2,65,1,52,6,61,1,61,1,65,1,46,4,61,0,10.0,8.0,7.5,67.0,49.0,54.0,59.0,52.0,63.0,65.0,NaN,0.00,-20.00,10.00,7.0,NaN,7,13,0,0,0,20,0,0,0,0,0,2200,46.0,8.000000,8.80,49.0,57.0,78.0,46.0,46.0,NaN,NaN,0,0,4,0,0,10,0,2,17,0,0,2,0,0,0,0,1,0,2,0,4,49.0
2,01/06/2023 2:00:00 PM,Yarmouth,1763,Turf,Good To Firm,Hcap,3,Fillyfudge,0,134,8.5,10.00,0,54,26,81,56,6,4,54,3,14.0,4,233,8,526,293,3,84,64,81,64,58,7,58,3,44,9,44,7,51,5,38,7,51,3,58,31,17.0,13.0,23.0,58.0,25.0,43.0,38.0,51.0,38.0,52.0,100.0,0.00,-7.00,14.29,226.0,186.0,4,1,0,2,0,5,0,0,0,0,0,3413,0.0,26.049999,25.46,25.0,80.0,80.0,0.0,0.0,NaN,NaN,0,0,1,0,0,2,0,1,3,0,1,2,0,0,1,0,3,0,1,0,1,25.0
3,01/06/2023 2:00:00 PM,Yarmouth,1763,Turf,Good To Firm,Hcap,4,Mudlahhim,0,135,6.0,7.60,0,55,64,60,42,10,1,55,2,7.5,2,338,2,582,244,9,57,66,69,52,68,1,58,3,58,1,58,3,58,3,58,3,0,5,0,0,5.5,6.5,5.5,76.0,61.0,58.0,62.0,61.0,61.0,72.0,NaN,8.11,-18.25,16.22,9.0,NaN,13,24,0,0,0,37,0,3,0,0,0,1576,59.0,2.000000,12.50,61.0,57.0,67.0,38.0,67.0,NaN,NaN,0,0,4,0,0,5,1,3,7,0,0,0,0,0,1,0,1,0,7,0,4,61.0
4,01/06/2023 2:00:00 PM,Yarmouth,1763,Turf,Good To Firm,Hcap,5,No Diggity,0,128,13.0,17.50,0,47,48,73,20,2,7,48,6,9.0,5,285,4,568,283,4,78,65,78,62,62,5,53,5,53,5,53,4,53,4,53,4,0,5,0,0,9.0,9.0,13.0,67.0,59.0,54.0,58.0,55.0,59.0,67.0,NaN,9.52,-14.93,14.29,9.0,NaN,5,37,0,0,0,42,0,4,0,0,0,1576,48.0,3.500000,32.00,59.0,53.0,78.0,52.0,30.0,NaN,NaN,0,0,3,1,4,14,4,6,38,0,0,1,0,0,1,0,0,0,1,0,3,59.0


In [3]:
bins = [800, 1000, 1200, 1400, 1600, 1800, 2000, 2200, 2400, 2600,
        2800, 3000, 3200, 3400, 3600, 3800, 4000]

df['distance_bucket'] = pd.cut(df['Distance (y)'], bins = bins,
                               labels=['800-1000', '1000-1200', '1200-1400', '1400-1600', '1600-1800', '1800-2000',
                                       '2000-2200', '2200-2400', '2400-2600', '2600-2800', '2800-3000',
                                       '3000-3200', '3200-3400', '3400-3600', '3600-3800', '3800-4000'])

In [4]:
df['evening_morning_price'] = (df['Morning Price'] / df['Evening Price'])

df['breakfast_morning_price'] = (df['Morning Price'] / df['Breakfast Price'])

In [5]:
df['weight'] = df['Weight (pounds)'] - df['Jockey Claim']

df['SP Odds Decimal1'] = df['BF Decimal SP']

In [6]:
list = df.columns.tolist()

In [7]:
remove = ['Race Time', 'Course', 'Distance (y)', 'Race Type', 'Official Going',
       'Classifications', 'Position', 'Horse', 'Jockey Claim',
       'Weight (pounds)','distance_bucket', 'SP Odds Decimal1']

list1 = [x for x in list if x not in remove]

In [8]:
normalized_df = DataCleaning.normalize_columns(df, list1)

X = normalized_df.drop(['Race Time', 'Course', 'Horse', 'Distance (y)', 'Race Type', 'Official Going', \
                        'Classifications', 'Position', 'Horse', 'Jockey Claim', 'Weight (pounds)', 'Won (1=Won, 0=Lost)', 'distance_bucket', 'SP Odds Decimal', \
                            'Evening Price', 'Breakfast Price', 'Morning Price', 'Main Stats', 'Won P/L Before'], axis = 1)
y = normalized_df[['Won (1=Won, 0=Lost)']]

X.fillna(0, inplace = True)

# Replace NaNs with 0 in numerical columns only
normalized_df[normalized_df.select_dtypes(include=['number']).columns] = normalized_df.select_dtypes(include=['number']).fillna(0)



/Users/andrewbarwise/miniforge3/envs/horse_trading/lib/python3.11/site-packages/sklearn/preprocessing/_data.py:480: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/Users/andrewbarwise/miniforge3/envs/horse_trading/lib/python3.11/site-packages/sklearn/preprocessing/_data.py:481: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


In [9]:
normalized_df.head()

,Race Time,Course,Distance (y),Race Type,Official Going,Classifications,Position,Horse,Jockey Claim,Weight (pounds),SP Odds Decimal,BF Decimal SP,"Won (1=Won, 0=Lost)",PRC Average,PRC Last Run,PRC 2nd Last Run,PRC 3rd Last Run,Pace Rating,Pace Rating Rank,OR,OR Rank,Tissue Odds Decimal,PFR Rank,Main,Main Rank,Main Stats,Stats,Stats Rank,Trn Stats,Jky Stats,TrnJky Stats,Hrs Stats,HA Career Speed Rating,HA Career Speed Rating Rank,HA Last 1 Year Speed Rating,HA Last 1 Year Speed Rating Rank,MR Career Speed Rating,MR Career Speed Rating Rank,MR Last 1 Year Speed Rating,MR Last 1 Year Speed Rating Rank,MR Last 3 Runs Speed Rating,MR Last 3 Runs Speed Rating Rank,LTO Speed Rating,LTO Speed Rating Rank,2nd LTO Speed Rating,2nd LTO Speed Rating Rank,3rd LTO Speed Rating,4th LTOt Speed Rating,Evening Price,Breakfast Price,Morning Price,dyHA,dyLTO,dyAVG,dyAVG(x),dyMR,dyMR(x),dy2HA,Finished Races SR,WON SR Before,Won P/L Before,Plc SR Before,DSLR(F),DSLR(J),FLAT TURF Runs Before,FLAT AW Runs Before,CHASE Runs Before,HURDLE Runs Before,NHF Runs Before,ALL FLAT Runs Before,FLAT TURF Wins Before,FLAT AW Wins Before,CHASE Wins Before,HURDLE Wins Before,NHF Wins Before,Distance In Yards LTO,Official Rating LTO,Distance To Winner LTO,Betfair SP Odds LTO (Decimcal),Speed Rating LTO,PRC Average LTO,PRC Last Run LTO,PRC 2nd Last Run LTO,PRC 3rd Last Run LTO,JSH Mean Movement Rank,Cherry Picked Rating,Today's Going Wins,Today's Going Places,Today's Going Runs,Today's Distance Wins,Today's Distance Places,Today's Distance Runs,Today's Class Wins,Today's Class Places,Today's Class Runs,Today's Course Wins,Today's Course Places,Today's Course Runs,Heavy/Soft Ground Wins,Heavy/Soft Ground Places,Heavy/Soft Ground Runs,Good to Soft Ground Wins,Good to Soft Ground Runs,Good Ground Wins,Good Ground Runs,Good to Firm Ground Wins,Good to Firm Ground Runs,SpeedRating LTO,distance_bucket,evening_morning_price,breakfast_morning_price,weight,SP Odds Decimal1
0,01/06/2023 2:00:00 PM,Yarmouth,1763,Turf,Good To Firm,Hcap,1,Simply Gorgeous,0,136,0.015941,0.009820,1.0,0.393939,0.131313,0.515152,0.535354,0.500000,0.111111,0.437500,0.000000,0.033613,0.212121,0.784810,0.121212,0.825488,0.895792,0.000000,0.925466,0.962733,0.940397,0.874372,0.640449,0.212121,0.404494,0.212121,0.619048,0.151515,0.418605,0.212121,0.418605,0.212121,0.404494,0.212121,0.000000,0.121212,0.000000,0.000000,0.027767,0.019771,0.016503,0.707865,0.404494,0.630952,0.607143,0.666667,0.686047,0.697674,0.0,0.0769,0.195951,0.0769,0.003100,0.000000,0.043956,0.087379,0.0,0.000000,0.0,0.086667,0.0,0.071429,0.0,0.0,0.0,0.202542,0.35625,0.081890,0.008332,0.404494,0.606061,0.515152,0.535354,0.767677,0.0,0.0,0.0,0.071429,0.068182,0.058824,0.00,0.076923,0.076923,0.000000,0.066038,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.019608,0.0,0.088235,0.404494,1600-1800,0.104592,0.152591,0.521127,10.83
1,01/06/2023 2:00:00 PM,Yarmouth,1763,Turf,Good To Firm,Hcap,2,Bold And Loyal,0,126,0.014941,0.009820,0.0,0.434343,0.303030,0.292929,0.717172,0.500000,0.111111,0.359375,0.258065,0.029612,0.030303,0.824895,0.060606,0.729047,0.689379,0.212121,0.844720,0.795031,0.774834,0.683417,0.730337,0.030303,0.730337,0.000000,0.619048,0.151515,0.709302,0.000000,0.709302,0.000000,0.730337,0.000000,0.516854,0.090909,0.685393,0.000000,0.035769,0.023105,0.021504,0.752809,0.550562,0.642857,0.702381,0.619048,0.732558,0.755814,0.0,0.0000,0.145749,0.1000,0.000664,0.000000,0.076923,0.126214,0.0,0.000000,0.0,0.133333,0.0,0.000000,0.0,0.0,0.0,0.291236,0.28750,0.041995,0.008791,0.550562,0.575758,0.787879,0.464646,0.464646,0.0,0.0,0.0,0.000000,0.090909,0.000000,0.00,0.109890,0.000000,0.060606,0.160377,0.0,0.000000,0.054054,0.0,0.0,0.000000,0.0,0.045455,0.0,0.039216,0.0,0.117647,0.550562,1600-1800,0.104592,0.169425,0.380282,10.83
2,01/06/2023 2:00:00 PM,Yarmouth,1763,Turf,Good To Firm,Hcap,3,Fillyfudge,0,134,0.014941,0.008989,0.0,0.545455,0.262626,0.818182,0.565657,0.500000,0.111111,0.421875,0.064516,0.051621,0.090909,0.698312,0.212121

In [10]:
%%script false --no-raise-error

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X, y)

# Get feature importances
importances = rf_classifier.feature_importances_
feature_names = X.columns

# Sort feature importances in descending order
indices = np.argsort(importances)[::-1][:10]

# Plot feature importances
plt.figure(figsize=(10, 6))
plt.title("Feature Importances")
plt.bar(range(10), importances[indices], align="center")
plt.xticks(range(10), feature_names[indices], rotation=90)
plt.xlabel("Feature")
plt.ylabel("Feature Importance")
plt.show()

## look at thresholds around variance

In [11]:
%%script false --no-raise-error

# Remove features with very low variance
threshold = 0.01  # Customize the threshold for your use case
selector = VarianceThreshold(threshold=threshold)
X_reduced = selector.fit_transform(X)

# Get the indices of the features that were kept
support = selector.get_support(indices=True)

# Create a list of feature names for the reduced dataset
reduced_feature_names = X.columns[support]

# Train a RandomForest classifier on the reduced dataset
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_reduced, y)

# Get feature importances
importances = rf_classifier.feature_importances_

# Sort feature importances in descending order and select top 10
indices = np.argsort(importances)[::-1][:10]

# Plot feature importances for the top 10 features
plt.figure(figsize=(10, 6))
plt.title("Top 10 Feature Importances")
plt.bar(range(10), importances[indices], align="center")
plt.xticks(range(10), reduced_feature_names[indices], rotation=90)
plt.xlabel("Feature")
plt.ylabel("Feature Importance")
plt.show()


## Recursive Feature Elimination
Use RFE to reduce the feature space by removing the least important features recursively.

In [12]:
%%script false --no-raise-error

from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rfe = RFE(rf_classifier, n_features_to_select=10)  # Customize number of features
rfe.fit(X, y)
print("Selected features:", X.columns[rfe.support_])

## remove highly correlated features and then use RFE

In [13]:
%%script false --no-raise-error

corr_matrix = X.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find index of feature columns with correlation greater than 0.9
to_drop = [column for column in upper.columns if any(upper[column] > 0.9)]

df_reduced = X.drop(columns=to_drop)

to_drop

In [14]:
%%script false --no-raise-error

rf_classifier1 = RandomForestClassifier(n_estimators=100, random_state=42)
rfe1 = RFE(rf_classifier1, n_features_to_select=10)  # Customize number of features
rfe1.fit(df_reduced, y)
print("Selected features:", df_reduced.columns[rfe1.support_])

## modelling variable by variable

In [15]:
def train_random_forest_classifier(df, feature_columns, target_column, random_state=42):
    """
    Train a RandomForestClassifier on specified columns of a DataFrame.

    Parameters:
    - df (pd.DataFrame): The input DataFrame.
    - feature_columns (list): List of column names to be used as features.
    - target_column (str): The name of the column to be used as the target.
    - random_state (int): Random state for reproducibility.

    Returns:
    - model (RandomForestClassifier): The trained Random Forest model.
    - accuracy (float): Accuracy score on the test set.
    """


    # Split data into training and testing sets
    train_data, test_data = DataCleaning.split_data(df=df)

    X_train = train_data[feature_columns]
    y_train = train_data[target_column]

    X_test = test_data[feature_columns]
    y_test = test_data[target_column]

    # Initialize the RandomForestClassifier
    rf_classifier = RandomForestClassifier(n_estimators=100, random_state=random_state)

    # Train the model
    rf_classifier.fit(X_train, y_train)

    # Make predictions
    y_pred = rf_classifier.predict(X_test)

    test_data['model_preds'] = y_pred

    print_metrics(y_test, y_pred)

    return test_data

In [16]:
def log_reg(df, feature_columns, target_column):
    # Define cost matrix
    C_FN = 5  # Cost of false negative
    C_FP = 1  # Cost of false positive

    train_data, test_data = DataCleaning.split_data(df=df)

    X_train = train_data[feature_columns]
    y_train = train_data[target_column]
    
    X_train.fillna(0, inplace = True)

    X_test = test_data[feature_columns]
    y_test = test_data[target_column]

    X_test.fillna(0, inplace = True)

    # Assuming y_train has values 0 and 1
    weights = np.ones(y_train.shape[0])
    weights[y_train == 1] = C_FN
    weights[y_train == 0] = C_FP

    # Train logistic regression model with custom weights
    model = LogisticRegression()
    model.fit(X_train, y_train, sample_weight=weights)

    # Predictions
    y_predz = model.predict(X_test)

    test_data['model_preds'] = y_predz

    # Evaluation
    print_metrics(y_test, y_predz)

    return test_data

In [17]:
def train_xgboost_classifier(df, feature_columns, target_column, random_state=42):
    train_data, test_data = DataCleaning.split_data(df=df)

    X_train = train_data[feature_columns]
    y_train = train_data[target_column]
    
    X_train.fillna(0, inplace = True)

    X_test = test_data[feature_columns]
    y_test = test_data[target_column]

    X_test.fillna(0, inplace = True)

    # initiate the xgb classifier
    xgb_classifier = XGBClassifier(
        n_estimators = 100,
        learning_rate = 0.1,
        max_depth = 6,
        subsample = 0.8,
        colsample_bytree = 0.8,
        random_state = random_state,
        use_label_encoder = False,
        eval_metrics = 'logloss'
    )

    xgb_classifier.fit(X_train, y_train)

    y_pred = xgb_classifier.predict(X_test)
    test_data['model_preds'] = y_pred

    print_metrics(y_test, y_pred)

    return test_data

In [18]:
normalized_df.head()

,Race Time,Course,Distance (y),Race Type,Official Going,Classifications,Position,Horse,Jockey Claim,Weight (pounds),SP Odds Decimal,BF Decimal SP,"Won (1=Won, 0=Lost)",PRC Average,PRC Last Run,PRC 2nd Last Run,PRC 3rd Last Run,Pace Rating,Pace Rating Rank,OR,OR Rank,Tissue Odds Decimal,PFR Rank,Main,Main Rank,Main Stats,Stats,Stats Rank,Trn Stats,Jky Stats,TrnJky Stats,Hrs Stats,HA Career Speed Rating,HA Career Speed Rating Rank,HA Last 1 Year Speed Rating,HA Last 1 Year Speed Rating Rank,MR Career Speed Rating,MR Career Speed Rating Rank,MR Last 1 Year Speed Rating,MR Last 1 Year Speed Rating Rank,MR Last 3 Runs Speed Rating,MR Last 3 Runs Speed Rating Rank,LTO Speed Rating,LTO Speed Rating Rank,2nd LTO Speed Rating,2nd LTO Speed Rating Rank,3rd LTO Speed Rating,4th LTOt Speed Rating,Evening Price,Breakfast Price,Morning Price,dyHA,dyLTO,dyAVG,dyAVG(x),dyMR,dyMR(x),dy2HA,Finished Races SR,WON SR Before,Won P/L Before,Plc SR Before,DSLR(F),DSLR(J),FLAT TURF Runs Before,FLAT AW Runs Before,CHASE Runs Before,HURDLE Runs Before,NHF Runs Before,ALL FLAT Runs Before,FLAT TURF Wins Before,FLAT AW Wins Before,CHASE Wins Before,HURDLE Wins Before,NHF Wins Before,Distance In Yards LTO,Official Rating LTO,Distance To Winner LTO,Betfair SP Odds LTO (Decimcal),Speed Rating LTO,PRC Average LTO,PRC Last Run LTO,PRC 2nd Last Run LTO,PRC 3rd Last Run LTO,JSH Mean Movement Rank,Cherry Picked Rating,Today's Going Wins,Today's Going Places,Today's Going Runs,Today's Distance Wins,Today's Distance Places,Today's Distance Runs,Today's Class Wins,Today's Class Places,Today's Class Runs,Today's Course Wins,Today's Course Places,Today's Course Runs,Heavy/Soft Ground Wins,Heavy/Soft Ground Places,Heavy/Soft Ground Runs,Good to Soft Ground Wins,Good to Soft Ground Runs,Good Ground Wins,Good Ground Runs,Good to Firm Ground Wins,Good to Firm Ground Runs,SpeedRating LTO,distance_bucket,evening_morning_price,breakfast_morning_price,weight,SP Odds Decimal1
0,01/06/2023 2:00:00 PM,Yarmouth,1763,Turf,Good To Firm,Hcap,1,Simply Gorgeous,0,136,0.015941,0.009820,1.0,0.393939,0.131313,0.515152,0.535354,0.500000,0.111111,0.437500,0.000000,0.033613,0.212121,0.784810,0.121212,0.825488,0.895792,0.000000,0.925466,0.962733,0.940397,0.874372,0.640449,0.212121,0.404494,0.212121,0.619048,0.151515,0.418605,0.212121,0.418605,0.212121,0.404494,0.212121,0.000000,0.121212,0.000000,0.000000,0.027767,0.019771,0.016503,0.707865,0.404494,0.630952,0.607143,0.666667,0.686047,0.697674,0.0,0.0769,0.195951,0.0769,0.003100,0.000000,0.043956,0.087379,0.0,0.000000,0.0,0.086667,0.0,0.071429,0.0,0.0,0.0,0.202542,0.35625,0.081890,0.008332,0.404494,0.606061,0.515152,0.535354,0.767677,0.0,0.0,0.0,0.071429,0.068182,0.058824,0.00,0.076923,0.076923,0.000000,0.066038,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.019608,0.0,0.088235,0.404494,1600-1800,0.104592,0.152591,0.521127,10.83
1,01/06/2023 2:00:00 PM,Yarmouth,1763,Turf,Good To Firm,Hcap,2,Bold And Loyal,0,126,0.014941,0.009820,0.0,0.434343,0.303030,0.292929,0.717172,0.500000,0.111111,0.359375,0.258065,0.029612,0.030303,0.824895,0.060606,0.729047,0.689379,0.212121,0.844720,0.795031,0.774834,0.683417,0.730337,0.030303,0.730337,0.000000,0.619048,0.151515,0.709302,0.000000,0.709302,0.000000,0.730337,0.000000,0.516854,0.090909,0.685393,0.000000,0.035769,0.023105,0.021504,0.752809,0.550562,0.642857,0.702381,0.619048,0.732558,0.755814,0.0,0.0000,0.145749,0.1000,0.000664,0.000000,0.076923,0.126214,0.0,0.000000,0.0,0.133333,0.0,0.000000,0.0,0.0,0.0,0.291236,0.28750,0.041995,0.008791,0.550562,0.575758,0.787879,0.464646,0.464646,0.0,0.0,0.0,0.000000,0.090909,0.000000,0.00,0.109890,0.000000,0.060606,0.160377,0.0,0.000000,0.054054,0.0,0.0,0.000000,0.0,0.045455,0.0,0.039216,0.0,0.117647,0.550562,1600-1800,0.104592,0.169425,0.380282,10.83
2,01/06/2023 2:00:00 PM,Yarmouth,1763,Turf,Good To Firm,Hcap,3,Fillyfudge,0,134,0.014941,0.008989,0.0,0.545455,0.262626,0.818182,0.565657,0.500000,0.111111,0.421875,0.064516,0.051621,0.090909,0.698312,0.212121

In [20]:
feature_columns = ['BF Decimal SP', 'LTO Speed Rating Rank', 'PFR Rank','Main Rank', 'HA Career Speed Rating Rank', "Today's Class Wins", "Today's Going Wins"]
target_column= 'Won (1=Won, 0=Lost)'

# Split data into training and testing sets
train_data, test_data = DataCleaning.split_data(df=df)

X_train = train_data[feature_columns]
y_train = train_data[target_column]

X_test = test_data[feature_columns]
y_test = test_data[target_column]

# Initialize the RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_classifier.fit(X_train, y_train)

# Make predictions
y_pred = rf_classifier.predict(X_test)

test_data['model_preds'] = y_pred

print_metrics(y_test, y_pred)

profit_calculation(test_data)

with open('../models/rf_model.pkl', 'wb') as f:
    pickle.dump(rf_classifier, f)


Confusion Matrix: 
[[8046  134]
 [ 857   94]]
Accuracy: 0.8914686233709341
Precision: 0.41228070175438597
Recall: 0.09884332281808622
F1 score: 0.15945716709075486
ROC AUC score: 0.5412309523625884
Total number of bets: 228
Total return from betting £1.00 on each prediction where model_preds == 1: £39.40
Return per pound invested: £0.17
Model accuracy: 41.23%


In [ ]:
feature_columns = ['BF Decimal SP', 'LTO Speed Rating Rank', 'PFR Rank','Main Rank', 'HA Career Speed Rating Rank', "Today's Class Wins", "Today's Going Wins"]
target_column= 'Won (1=Won, 0=Lost)'


ab = train_xgboost_classifier(normalized_df, feature_columns, target_column)
print()
profit_calculation(ab)

In [ ]:
feature_columns = ['BF Decimal SP', 'LTO Speed Rating Rank']
target_column= 'Won (1=Won, 0=Lost)'


bb = log_reg(normalized_df, feature_columns, target_column)
print()
profit_calculation(bb)

In [ ]:
feature_columns = ['BF Decimal SP', 'Main']
target_column= ['Won (1=Won, 0=Lost)']


aa = train_random_forest_classifier(normalized_df, feature_columns, target_column)
print()
profit_calculation(aa)

In [ ]:
feature_columns = ['BF Decimal SP', 'Main', 'OR', 'Won P/L Before']
target_column= 'Won (1=Won, 0=Lost)'


bb = log_reg(normalized_df, feature_columns, target_column)
print()
profit_calculation(bb)

In [ ]:
feature_columns = ['BF Decimal SP', 'Won P/L Before', 'Main', 'Speed Rating LTO', 'dyHA']
target_column= 'Won (1=Won, 0=Lost)'


bb = log_reg(normalized_df, feature_columns, target_column)
print()
profit_calculation(bb)

In [ ]:
feature_columns = ['BF Decimal SP', 'Won P/L Before', 'Main', 'evening_morning_price','TrnJky Stats', 'dyHA']
target_column= 'Won (1=Won, 0=Lost)'


normalized_df[feature_columns] = normalized_df[feature_columns].fillna(0)


aa = train_random_forest_classifier(normalized_df, feature_columns, target_column)
print()
profit_calculation(aa)